<a href="https://www.kaggle.com/code/sameekshyapati/traffic-image-classification?scriptVersionId=184854024" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

In [ ]:
import numpy as np
import pandas as pd
import os
import pickle
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Input
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint


In [ ]:
# Path to the Kaggle input directory
input_dir = '/kaggle/input/signs-input/'

# Load the data
with open(os.path.join(input_dir, 'train.pickle'), 'rb') as f:
    train_data = pickle.load(f)
with open(os.path.join(input_dir, 'valid.pickle'), 'rb') as f:
    valid_data = pickle.load(f)
with open(os.path.join(input_dir, 'test.pickle'), 'rb') as f:
    test_data = pickle.load(f)

# Define categories
categories = pd.read_csv(os.path.join(input_dir, 'label_names.csv'))
num_classes = len(categories)


In [ ]:
# Unpack data
X_train, y_train = train_data['features'], train_data['labels']
X_valid, y_valid = valid_data['features'], valid_data['labels']
X_test, y_test = test_data['features'], test_data['labels']

# Resize images if needed (in this case, images are assumed to be 32x32)
# Convert labels to categorical
y_train = to_categorical(y_train, num_classes)
y_valid = to_categorical(y_valid, num_classes)
y_test = to_categorical(y_test, num_classes)

# Normalize images
X_train = X_train / 255.0
X_valid = X_valid / 255.0
X_test = X_test / 255.0

In [ ]:
from sklearn.utils import shuffle
X_train, y_train = shuffle(X_train, y_train)

In [ ]:
# Define the CNN model
model = Sequential([
    Input(shape=(32, 32, 3)),
    Conv2D(32, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Dropout(0.25),
    
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(num_classes, activation='softmax')
])

# Compile the model
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Set up callbacks
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
model_checkpoint = ModelCheckpoint('best_model.keras', save_best_only=True, monitor='val_loss')

# Train the model
history = model.fit(X_train, y_train,
                    validation_data=(X_valid, y_valid),
                    epochs=30, batch_size=32,
                    callbacks=[early_stopping, model_checkpoint])


In [ ]:
# Load the best model
model.load_weights('best_model.keras')

# Evaluate on the test set
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f'Test Accuracy: {test_acc * 100:.2f}%')


In [ ]:
def display_image_with_prediction(index):
    plt.imshow(X_test[index])
    actual_class = np.argmax(y_test[index])
    predicted_class = np.argmax(model.predict(X_test[index:index+1]))
    plt.title(f"Actual: {categories['SignName'][actual_class]}, Predicted: {categories['SignName'][predicted_class]}")
    plt.axis('off')
    plt.show()

# Display a few sample predictions
for i in range(5):
    display_image_with_prediction(i)
